## DIA 047: Implementacion de Notificaciones por Correo para Errores Criticos

En el Día 47, se implementa un sistema de notificaciones por correo electrónico para alertar a los administradores cuando se produzca un error crítico en la API. Este mecanismo se integra en el manejador de errores de Flask y utiliza Flask-Mail para enviar automáticamente un correo a la dirección de administración configurada. Esto permite una rápida intervención y resolución de incidencias, mejorando la seguridad y la confiabilidad del sistema.

Código Completo (api.py)
python
Copiar
import os
import io
import random
import json
import time
import threading
import logging
import hashlib
from datetime import datetime
from functools import wraps

import requests
import redis
import numpy as np
from PIL import Image

from flask import Flask, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room
from flasgger import Swagger

# Configuración básica y variables de entorno
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Configuración de Flasgger para documentación
swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": "apispec_1",
            "route": "/apispec_1.json",
            "rule_filter": lambda rule: True,
            "model_filter": lambda tag: True,
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}
swagger = Swagger(app, config=swagger_config)

# Inicialización de extensiones
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])
socketio = SocketIO(app, cors_allowed_origins="*")

# Configuración de Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Inicialización de Redis
redis_client = redis.Redis(
    host=os.getenv("REDIS_HOST", "localhost"),
    port=int(os.getenv("REDIS_PORT", 6379)),
    db=0
)

# ---------------------------
# Modelos
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Otros campos omitidos para este ejemplo

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# Nuevo Modelo: AuditLog para auditoría
class AuditLog(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    action = db.Column(db.String(120), nullable=False)
    details = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "action": self.action,
            "details": self.details,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Función para Registro de Auditoría
# ---------------------------
def log_audit_event(username, action, details=""):
    audit = AuditLog(username=username, action=action, details=details)
    db.session.add(audit)
    db.session.commit()
    logger.info(f"Audit log: {username} - {action} - {details}")

# ---------------------------
# Decorador para Roles (simplificado)
# ---------------------------
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            if getattr(user, 'role', 'user') != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Función para Enviar Notificaciones por Correo en Caso de Error Crítico
# ---------------------------
def send_error_email(subject, body):
    admin_email = os.getenv("ADMIN_EMAIL", "admin@example.com")
    msg = Message(subject=subject, recipients=[admin_email], body=body)
    try:
        mail.send(msg)
        logger.info("Correo de alerta enviado a %s", admin_email)
    except Exception as e:
        logger.error("Error enviando correo de alerta: %s", str(e))

# ---------------------------
# Manejador de Errores Críticos
# ---------------------------
@app.errorhandler(500)
def internal_error(error):
    error_message = f"Error Crítico: {str(error)}"
    send_error_email("Error Crítico en la API", error_message)
    logger.exception("Error interno: %s", error)
    return jsonify({"error": "Internal Server Error"}), 500

# ---------------------------
# Endpoints Comunes
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    access_token = create_access_token(identity=username)
    log_audit_event(username, "login", "Usuario inició sesión")
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=access_token), 200

@app.route('/health', methods=['GET'])
def health():
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoint de Predicción con Caching Avanzado (v1)
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400

    file_bytes = file.read()
    cache_key = hashlib.md5(file_bytes).hexdigest()
    cached_result = redis_client.get(cache_key)
    if cached_result:
        logger.info("Resultado obtenido de la caché para la clave: %s", cache_key)
        result = json.loads(cached_result)
        result['cached'] = True
        log_audit_event(get_jwt_identity(), "predict_v1", "Resultado obtenido de la caché")
        return jsonify(result), 200

    image = Image.open(io.BytesIO(file_bytes)).convert('L')
    image = image.resize((28, 28))
    image_array = np.array(image).astype('float32') / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    image_array = np.expand_dims(image_array, axis=-1)
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1", "cached": False}
    redis_client.setex(cache_key, 600, json.dumps(result))
    logger.info("v1: Predicción realizada y almacenada en caché con clave: %s", cache_key)
    log_audit_event(get_jwt_identity(), "predict_v1", "Predicción procesada y cacheada")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')

# ---------------------------
# Ejecutar la aplicación con soporte para WebSockets (opcional) y manejo de errores críticos
# ---------------------------
if __name__ == '__main__':
    socketio.run(app, debug=True)
Explicación del Código
Configuración y Extensiones:

Se configura la aplicación Flask, incluyendo JWT, SQLAlchemy, Flask-Mail, Flask-Limiter, y Flask-SocketIO.
Se configura Flasgger para la documentación automática de la API.
Se inicializa un cliente Redis para el caching avanzado.
Se configura logging para registrar eventos y errores.
Modelos:

Se definen los modelos User, Feedback y un nuevo modelo AuditLog para registrar eventos de auditoría.
Función de Auditoría:

log_audit_event registra eventos críticos en la base de datos, facilitando la trazabilidad.
Manejador de Errores Críticos:

Se implementa un manejador de errores para el código 500, que envía un correo de alerta a un administrador utilizando Flask-Mail.
La función send_error_email es llamada cuando ocurre un error crítico, notificando a través del correo electrónico configurado en la variable ADMIN_EMAIL.
Endpoints:

/login: Permite iniciar sesión y obtener un token JWT.
/health: Un endpoint de health check.
/api/v1/predict: Implementa un sistema de caching avanzado utilizando Redis:
Calcula una clave única (hash MD5) a partir del contenido del archivo.
Verifica si el resultado ya está en caché y lo retorna, o procesa la imagen, simula la predicción y guarda el resultado en Redis.
Se registran eventos de auditoría y logs en cada acción crítica.
Ejecución:

La aplicación se ejecuta con soporte para WebSockets (Flask-SocketIO) para futuras funcionalidades en tiempo real.
